# Module

In [49]:
import os
import glob
import shutil
import zipfile

import warnings
warnings.filterwarnings('ignore')
import subprocess
from tqdm import tqdm
from time import time
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta


import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.image as mpimg
import matplotlib.colors as mcolors

import ee
ee.Authenticate()
ee.Initialize(project='ee-21hs0878')

import rasterio
import geopandas as gpd
from osgeo import gdal, osr
from pyproj import Transformer
from shapely.ops import unary_union, polygonize
from shapely.geometry import box, shape, Point, Polygon, MultiPoint, MultiPolygon, LineString, MultiLineString

import landfire

from util import *

# Function


In [3]:
def extract_band_data(image):
    # Create a feature with the extracted data as properties
    band_data = image.reduceRegion(
        reducer=ee.Reducer.mean(),  # Change reducer if needed (e.g., sum, median)
        geometry=point,
        scale=1000  # Set the resolution in meters
    )
    # Return a Feature with the extracted data as properties
    return ee.Feature(None, band_data)


def convert_id_to_datetime(ids):
    # Check if input is a list; if not, wrap it in a list
    if isinstance(ids, str):
        ids = [ids]

    # Process each ID in the list
    result = []
    for id_string in ids:
        # Extract components
        year = int(id_string[:4])
        month = int(id_string[4:6])
        day = int(id_string[6:8])
        hour = int(id_string[8:10])
        forecast_offset = int(id_string[11:])  # After 'F'

        # Base datetime
        base_datetime = datetime(year, month, day, hour)

        # Add forecast offset
        forecast_datetime = base_datetime + timedelta(hours=forecast_offset)

        result.append(forecast_datetime)

    return result

def fju_weather(gdf, fireid, period):
    # Load the ImageCollection and filter by date
    ex = gdf[gdf['fireID'] == fireid].copy()
    clat, clon = ex.iloc[-1, 2:4]
    global point
    point = ee.Geometry.Point([clon, clat])

    start_time = ex.iloc[period,1]

    try:
        end_time = ex.iloc[period+1,1]
    except:
        end_time = start_time + timedelta(hours=12)

    start_date = (end_time- timedelta(days=3)).strftime("%Y-%m-%d")
    end_date = end_time.strftime("%Y-%m-%d")

    dataset = ee.ImageCollection('NOAA/GFS0P25').filter(
        ee.Filter.date(start_date, end_date)
    )
    # Apply the extraction function to each image in the dataset
    band_values = dataset.map(extract_band_data)

    # Get the results as a list of dictionaries
    band_values_list = band_values.getInfo()['features']

    columns = ["id",'relative_humidity_2m_above_ground', 'temperature_2m_above_ground', 'total_cloud_cover_entire_atmosphere', \
                        'total_precipitation_surface','u_component_of_wind_10m_above_ground', 'v_component_of_wind_10m_above_ground']
    columns = {c:[] for c in columns}

    for result in band_values_list:
        columns['id'].append(result['id'])
        i = ['id']
        for key, value in result['properties'].items():
            if key in columns.keys():
                columns[key].append(value)
                i.append(key)
        for key in columns.keys():
            if key not in i:
                columns[key].append(np.nan)
    # Print the results
    weather_stream = pd.DataFrame(columns)
    weather_stream.columns = ['id', 'RH', 'Temp', 'CloudCov', 'HrlyPcp', 'Wind_U', 'Wind_V']
    weather_stream['id'] = convert_id_to_datetime(weather_stream['id'])

    final_weather = {c:[] for c in ['Datetime', 'RH', 'Temp', 'CloudCov', 'HrlyPcp', 'Wind_U', 'Wind_V']}
    current_time = start_time
    while current_time <= end_time:
        final_weather['Datetime'].append(current_time)
        iter_data = weather_stream[weather_stream['id'] == str(current_time)].iloc[:, 1:].copy()

        plug_value = iter_data.median()

        if weather_stream[weather_stream['id'] == str(current_time)].iloc[0,-1] % 6 == 0:
            for c in  ['RH', 'Temp', 'Wind_U', 'Wind_V']:
                final_weather[c].append(iter_data.iloc[-1][c])
            final_weather['CloudCov'].append(plug_value['CloudCov'])
            final_weather['HrlyPcp'].append(plug_value['HrlyPcp'])

        else:
            for c in ['RH', 'Temp', 'CloudCov', 'HrlyPcp', 'Wind_U', 'Wind_V']:
                final_weather[c].append(plug_value[c])

        current_time += timedelta(hours=1)
    final_weather = pd.DataFrame(final_weather)
    final_weather['WindSpd'] = np.sqrt(final_weather['Wind_U']**2 + final_weather['Wind_V']**2)
    final_weather['WindDir'] = (270 - np.degrees(np.arctan2(final_weather['Wind_U'], final_weather['Wind_V']))) % 360
    final_weather['Time'] = pd.to_datetime(final_weather['Datetime'])
    
    final_weather['Year'] = final_weather['Datetime'].dt.year
    final_weather['Mth'] = final_weather['Datetime'].dt.month
    final_weather['Day'] = final_weather['Datetime'].dt.day
    final_weather['Hour'] = final_weather['Datetime'].dt.hour
    final_weather['Min'] = final_weather['Datetime'].dt.minute
    final_weather['Hour'] = final_weather['Hour'].apply(lambda x: f"{x:02d}")
    final_weather['Min'] = final_weather['Min'].apply(lambda x: f"{x:02d}")
    final_weather['Time'] = final_weather['Hour'] + final_weather['Min']

    final_weather['Temp'] = final_weather['Temp'] * 9/5 + 32         # Celsius to Fahrenheit
    final_weather['HrlyPcp'] = final_weather['HrlyPcp'] / 25.4       # Millimeters to Inches
    final_weather['WindSpd'] = final_weather['WindSpd'] / 1.609      # km/h to mph

    final_weather = final_weather[['Year', 'Mth', 'Day', 'Time', 'Temp', 'RH', 'HrlyPcp', 'WindSpd', 'WindDir', 'CloudCov']]
    return final_weather

def plot_geometry(geom, edge_color='black', face_color='cyan'):
    """Plot a Shapely geometry which can be a Polygon or MultiPolygon."""
    # If geometry is a Polygon, plot it directly.
    if geom.geom_type == 'Polygon':
        x, y = geom.exterior.xy
        plt.plot(x, y, color=edge_color)
        plt.fill(x, y, color=face_color, alpha=0.5)
    # If geometry is a MultiPolygon, iterate over each component.
    elif geom.geom_type == 'MultiPolygon':
        for poly in geom.geoms:
            plot_geometry(poly, edge_color=edge_color, face_color=face_color)

def find_square_boundaries(centroid, width_m):
    """Computes the boundaries of a square in latitude and longitude.

    Args:
        centroid (tuple): A tuple containing the latitude and longitude of the centroid.
        width_m (float): The width of the square in meters.

    Returns:
        tuple: (top, bottom, right, left) boundaries in degrees.
    """
    earth_radius = 6_371_000.0
    lat, lon = centroid
    lat_rad = np.deg2rad(lat)
    half_ang_lat = (width_m / 2) / earth_radius
    half_ang_lon = (width_m / 2) / (earth_radius * np.cos(lat_rad))
    top = lat_rad + half_ang_lat
    bottom = lat_rad - half_ang_lat
    right = np.deg2rad(lon) + half_ang_lon
    left = np.deg2rad(lon) - half_ang_lon

    return (
        np.rad2deg(top),
        np.rad2deg(bottom),
        np.rad2deg(right),
        np.rad2deg(left),
    )

def perimeter_input(gdf, fireid:int, period:int, timestep = 60, distance_resolution = 30, \
                    perimeter_resolution = 60, min_ignition_distance = 15, spot_grid_resolution = 15,\
                    spot_probability = 0, spot_ignition_delay = 0, min_spot_distance = 30, \
                    acceleration_on = 1, fill_barriers = 0, spotting_seed = 114514, raws_units = 'English',\
                    num_processors = 1):
    gdf = gdf[gdf['fireID'] == fireid]
    gdf.reset_index(drop = True, inplace = True)

    clat, clon = gdf.iloc[-1, 2:4]
    point = ee.Geometry.Point([clon, clat])

    datetime_format =  "'%m %d %H%M'"
    frequency = 'H' # Hourly frequency
    start_time = gdf.iloc[period,1]
    try:
        end_time = gdf.iloc[period+1,1]
    except:
        end_time = start_time + pd.Timedelta(hours=12)
    burn_last = (pd.to_datetime(end_time, format=datetime_format) - pd.Timedelta(minutes=1)).strftime(datetime_format)
    burn_periods_data = start_time.strftime(datetime_format) + ' ' + burn_last[-5:]
    burn_periods_count = 1

    weather_df = fju_weather(gdf, fireid, period)

    raws_count = weather_df.shape[0]
    # weather_df
    formatted_lines = []
    for row in range(raws_count):
        row = weather_df.iloc[row, :] # Get the current row's data

        # Format the current row's data into a string
        line_data = (
            f"{row['Year']} {row['Mth']} {row['Day']:02} {row['Time']} "
            f"{row['Temp']:.0f} {row['RH']:.0f} {row['HrlyPcp']:.0f} {row['WindSpd']:.0f} "
            f"{row['WindDir']:.0f} {row['CloudCov']:.0f}\n"
        )
        formatted_lines.append(line_data)

        # Join all formatted lines into a single string
    raws_data = "".join(formatted_lines)

    image = ee.Image('USGS/SRTMGL1_003')

    # Extract elevation at the point
    elevation = image.sampleRegions(
        collection=ee.FeatureCollection([point]),
        scale=30
    )

    # Print the result
    elevation = elevation.getInfo()['features'][0]['properties']['elevation']  * 3.28084

    start_time = start_time.strftime(datetime_format)
    end_time = end_time.strftime(datetime_format)
    farsite_template = \
    f"""\
FARSITE INPUTS FILE VERSION 1.0
FARSITE_START_TIME: {start_time}
FARSITE_END_TIME: {end_time}
FARSITE_TIMESTEP: {timestep}
FARSITE_DISTANCE_RES: {distance_resolution}
FARSITE_PERIMETER_RES: {perimeter_resolution}
FARSITE_MIN_IGNITION_VERTEX_DISTANCE: {min_ignition_distance}
FARSITE_SPOT_GRID_RESOLUTION: {spot_grid_resolution}
FARSITE_SPOT_PROBABILITY: {spot_probability}
FARSITE_SPOT_IGNITION_DELAY: {spot_ignition_delay}
FARSITE_MINIMUM_SPOT_DISTANCE: {min_spot_distance}
FARSITE_ACCELERATION_ON: {acceleration_on}
FARSITE_FILL_BARRIERS: {fill_barriers}
SPOTTING_SEED: {spotting_seed}
FARSITE_BURN_PERIODS: {burn_periods_count}
{burn_periods_data}

FUEL_MOISTURES_DATA: 25
0 6 7 8 60 90
91 6 7 8 60 90
93 6 7 8 60 90
98 6 7 8 60 90
99 6 7 8 60 90
101 6 7 8 60 90
102 6 7 8 60 90
104 6 7 8 60 90
121 6 7 8 60 90
122 6 7 8 60 90
141 6 7 8 60 90
142 6 7 8 60 90
143 6 7 8 60 90
145 6 7 8 60 90
147 6 7 8 60 90
1 6 7 8 60 90
2 6 7 8 60 90
5 6 7 8 60 90
181 6 7 8 60 90
182 6 7 8 60 90
183 6 7 8 60 90
185 6 7 8 60 90
186 6 7 8 60 90
188 6 7 8 60 90
189 6 7 8 60 90
RAWS_ELEVATION: {elevation}
RAWS_UNITS: {raws_units}

RAWS: {raws_count}
{raws_data}
FOLIAR_MOISTURE_CONTENT: 100
CROWN_FIRE_METHOD: Finney
NUMBER_PROCESSORS: {num_processors}


FLAMELENGTH:
SPREADRATE:
INTENSITY:
CROWNSTATE:
    """
    file_path = rf"{fireid}\{period}\para.input"
    farsite_template = farsite_template.replace("'", "")
    # Write the content to the file
    try:
        with open(file_path, 'w') as f:
            f.write(farsite_template)
        print(f"Successfully wrote FARSITE input to: {file_path}")
    except IOError as e:
        print(f"Error writing file {file_path}: {e}")
    except KeyError as e:
        print(f"Error: Missing parameter {e} in the params dictionary.")
    return farsite_template

# Main

In [ ]:
gdf = gpd.read_file(r"LargeFires_2012-2020.gpkg")
gdf = gdf[['fireID', 'time', 'clat', 'clon', 'farea', 'geometry']]
gdf['time'] = pd.to_datetime(gdf['time'])

time_limit = gdf[['fireID', 'time']].copy()
time_limit = time_limit[time_limit['time'] < '2016/07/01']
time_limit = time_limit['fireID'].unique()

duration_filter = gdf.groupby('fireID').count().reset_index().copy()
duration_filter = duration_filter[(duration_filter['time'] > 10) & (duration_filter['time'] < 20)]['fireID'].to_list()

area_filter = gdf[['fireID', 'farea']].copy()
area_filter = area_filter[area_filter.duplicated(keep=False)]
area_filter = area_filter['fireID'].unique()

multipolygon_fireIDs = set(gdf[gdf.geometry.geom_type == 'MultiPolygon']['fireID'].unique())

gdf = gdf[(~gdf['fireID'].isin(time_limit))
        & (gdf['fireID'].isin(duration_filter))
        & (~gdf['fireID'].isin(area_filter))
        & (~gdf['fireID'].isin(multipolygon_fireIDs))]

# fireid_list = gdf['fireID'].unique()
fireid_list = [3173]
for fireid in fireid_list:
    # create_folder(str(fireid))
    
    temp_df = gdf[gdf['fireID'] == fireid].copy()
    clat, clon = temp_df.iloc[-1, 2:4]

    SQUARE_WIDTH_M = 40000
    LANDFIRE_LAYERS = [
        "ELEV2020", "SLPD2020", "ASP2020",
        "220F40_22", "220CC_22", "220CH_22",
        "220CBH_22", "220CBD_22"
    ]
    OUTPUT_ZIP_FILE = "lf_data.zip"
    OUTPUT_RASTER_DIR = "lf_rasters"

    top, bottom, right, left = find_square_boundaries((clat, clon), SQUARE_WIDTH_M)
    
    bbox_str = f"{left} {bottom} {right} {top}"  # min_x min_y max_x max_y
    print(bbox_str)

    break
    lf = landfire.Landfire(bbox=bbox_str)
    lf.request_data(
        layers=LANDFIRE_LAYERS,
        output_path=OUTPUT_ZIP_FILE
    )

    os.makedirs(OUTPUT_RASTER_DIR, exist_ok=True)
    with zipfile.ZipFile(OUTPUT_ZIP_FILE, "r") as z:
        z.extractall(OUTPUT_RASTER_DIR)

    tif_list = glob.glob(os.path.join(OUTPUT_RASTER_DIR, "*.tif"))
    if not tif_list:
        raise RuntimeError(f"No .tif files found in {OUTPUT_RASTER_DIR}")
    tif_path = tif_list[0]
    print(f"✅ First LANDFIRE raster found: {tif_path}")

    gdal.AllRegister()
    output_file = rf'{fireid}\{fireid}.lcp'

    ds = gdal.Open(tif_path)
    if ds is None:
        print(f"Unable to open {tif_path}")
    else:
        # Get the GDAL driver for LCP format
        driver = gdal.GetDriverByName('LCP')
        if driver is None:
            print("LCP driver is not available in your GDAL build.")
        else:
            # Optional: set creation options (adjust these as needed for your data)
            creation_options = [
                'ELEVATION_UNIT=METERS',      # Can also be FEET if applicable
                'SLOPE_UNIT=DEGREES',        # Use PERCENT if needed
                'ASPECT_UNIT=AZIMUTH_DEGREES', # Other valid values: GRASS_CATEGORIES, etc.
                # 'FUEL_MODEL_OPTION=NO_CUSTOM_AND_NO_FILE',  # Uncomment if needed
            ]

            # Create a copy in the LCP format
            out_ds = driver.CreateCopy(output_file, ds, options=creation_options)
            if out_ds is None:
                print("Conversion to LCP failed.")
            else:
                print(f"Conversion to LCP successful: {output_file}")
                out_ds = None # Close the output dataset
        ds = None # Close the input dataset

    
    # Open the first raster to get its CRS
    ds = gdal.Open(output_file)
    proj = ds.GetProjection()
    srs = osr.SpatialReference()
    srs.ImportFromWkt(proj)

    crs_proj = srs.ExportToProj4()
    ds = None  # Close the dataset
    period, _ = temp_df.shape
    # Fire boundary processing
    for i in range(period):
        current_folder = rf'{fireid}\{i}'
        create_folder(current_folder)
        OUTPUT_SHP_DIR = rf"{fireid}\{i}\fire_boundary_albers"
        OUTPUT_SHP_FILE = rf"{fireid}\{i}\fire_boundary.shp"
        
        shp_geom = temp_df.iloc[i, -1]

        fire_boundary_gdf = gpd.GeoDataFrame(geometry=[shp_geom], crs="EPSG:4326")
        fire_boundary_gdf = fire_boundary_gdf.to_crs(crs_proj)  # Use PROJ string directly
        fire_boundary_gdf.to_file(OUTPUT_SHP_FILE)
        # Save reprojected shapefile
        perimeter_input(gdf, fireid, i)

    if os.path.isfile(OUTPUT_ZIP_FILE):
        os.remove(OUTPUT_ZIP_FILE)
        print(f"Deleted file: {OUTPUT_ZIP_FILE}")
    else:
        print(f"No such file to delete: {OUTPUT_ZIP_FILE}")
    if os.path.isdir(OUTPUT_RASTER_DIR):
        shutil.rmtree(OUTPUT_RASTER_DIR)
        print(f"Deleted directory and all contents: {OUTPUT_RASTER_DIR}")
    else:
        print(f"No such directory to delete: {OUTPUT_RASTER_DIR}")

-122.91400816630518 38.37926961378655 -122.45397667451162 38.73899825615404


In [73]:
gdf = gpd.read_file(r"LargeFires_2012-2020.gpkg", engine = 'pyogrio')
gdf = gdf[['fireID', 'time', 'clat', 'clon', 'farea', 'geometry']]
gdf['time'] = pd.to_datetime(gdf['time'])

time_limit = gdf[['fireID', 'time']].copy()
time_limit = time_limit[time_limit['time'] < '2016/07/01']
time_limit = time_limit['fireID'].unique()

duration_filter = gdf.groupby('fireID').count().reset_index().copy()
duration_filter = duration_filter[(duration_filter['time'] > 10) & (duration_filter['time'] < 20)]['fireID'].to_list()

area_filter = gdf[['fireID', 'farea']].copy()
area_filter = area_filter[area_filter.duplicated(keep=False)]
area_filter = area_filter['fireID'].unique()

multipolygon_fireIDs = set(gdf[gdf.geometry.geom_type == 'MultiPolygon']['fireID'].unique())

gdf = gdf[(~gdf['fireID'].isin(time_limit))
        & (gdf['fireID'].isin(duration_filter))
        & (~gdf['fireID'].isin(area_filter))
        & (~gdf['fireID'].isin(multipolygon_fireIDs))]

fireid = 1819

temp_df = gdf[gdf['fireID'] == fireid].reset_index().copy()

gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

EPSG:4326
